<a href="https://colab.research.google.com/github/pepperamy/others/blob/main/confcall_orgranize_newtranscript_step3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import re

In [23]:
from google.colab import drive
drive.mount('/content/drive')

import sys, os
cur_path = os.path.join('/content/drive/MyDrive/','Conf_Call')
os.chdir(cur_path)
print(os.getcwd())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/.shortcut-targets-by-id/1T3UFPXclB4txEq9f4wIp4I_gdwU6NmzO/Conf_Call


In [24]:
!pip install tika
from tika import parser

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [25]:
# use fuzzy match to remove the notes
!pip install fuzzywuzzy
from fuzzywuzzy import fuzz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# import path data

In [9]:
df = pd.read_csv('/content/drive/MyDrive/Conf_Call/code/data/Conf_features/add_new_transcript_v3.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (7,13,45) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [10]:
df.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,fyearq,fqtr,permno,path,report_day,gvkey,tic,...,num_question_analyst,num_analyst,num_question,sen_count,sen_count_large,sen_count_large/analyst,sen_count_large/question_analyst,sen_count_large/question,sent_per_question_analyst,cik
0,0.0,0.0,0.0,2015,1,13576,/backup_disk/conference_calls/BSMX/3112566-gru...,20150429,243588.0,BSMX,...,3.333333,3.0,10.0,11.666667,29.0,9.666667,0.966667,2.900000,3.111111,NaN
1,1.0,1.0,1.0,2013,2,92988,/backup_disk/conference_calls/CFN/1167001-care...,20130207,183257.0,CFN,...,2.500000,12.0,30.0,11.000000,115.0,9.583333,0.319444,3.833333,3.805556,NaN
2,2.0,2.0,2.0,2014,2,31077,/backup_disk/conference_calls/CUB/2213213-cubi...,20140512,3639.0,CUB,...,3.250000,4.0,13.0,8.500000,26.0,6.500000,0.500000,2.000000,2.125000,NaN
3,3.0,3.0,3.0,2015,2,31077,/backup_disk/conference_calls/CUB/3193576-cubi...,20150518,3639.0,CUB,...,2.500000,4.0,10.0,6.750000,17.0,4.250000,0.425000,1.700000,2.083333,NaN
4,4.0,4.0,4.0,2014,3,89252,/backup_disk/conference_calls/FVE/2762725-five...,20141216,146500.0,FVE,...,2.500000,2.0,5.0,8.500000,14.0,7.000000,1.400000,2.800000,2.750000,NaN


In [11]:
df_1 = df[[ 'fyearq','fqtr', 'permno', 'path',
 'report_day', 'gvkey','tic','encoded_unique_ticker_ts','cik']]

In [12]:
list(df.columns)

['Unnamed: 0',
 'Unnamed: 0.1',
 'Unnamed: 0.1.1',
 'fyearq',
 'fqtr',
 'permno',
 'path',
 'report_day',
 'gvkey',
 'tic',
 'encoded_unique_ticker_ts',
 'fyear_by_rule',
 'fqtr_by_rule',
 'special_call',
 'LPERMNO',
 'fyr',
 'roaq',
 'leverage',
 'datadate',
 'indfmt',
 'consol',
 'popsrc',
 'datafmt',
 'curcdq',
 'txditcq',
 'prccq',
 'sic',
 'size',
 'BE0',
 'BE',
 'BM',
 'actual_p',
 'expected_p',
 'actual_d',
 'expected_d',
 'deflator1',
 'suep1',
 'surp',
 'actual_p_2',
 'expected_p_2',
 'actual_d_2',
 'expected_d_2',
 'deflator2',
 'suep2',
 'sued2',
 'rdq',
 'bm_w',
 'leverage_w',
 'leverage_w2',
 'roa_w',
 'fyearqrt',
 'gap',
 'car_-1_1',
 'car_-1_7',
 'car_-1_5',
 'car_-1_14',
 'car_-1_30',
 'car_-1_60',
 'car_0_1',
 'car_0_7',
 'car_0_5',
 'car_0_14',
 'car_0_30',
 'car_0_60',
 'car_2_7',
 'car_2_14',
 'car_2_30',
 'car_2_60',
 'car_7_14',
 'car_7_30',
 'car_7_60',
 'net_hny',
 'net_lm',
 'net_harvard',
 'word_num',
 'uncertainty',
 '%uncertain',
 'SP_1',
 'SP_2',
 'SP_3',
 

In [13]:
df_1.shape

(44412, 9)

In [14]:
df_1.isnull().sum()

fyearq                          0
fqtr                            0
permno                          0
path                            0
report_day                      0
gvkey                        1585
tic                             0
encoded_unique_ticker_ts     2978
cik                         43763
dtype: int64

In [15]:
df_2 = df_1[df_1.path.str.contains('newly_add_transcript')]

In [16]:
df_2.head()

,fyearq,fqtr,permno,path,report_day,gvkey,tic,encoded_unique_ticker_ts,cik
42175,2016,3,91616,./code/data/newly_add_transcript/Transcript_Di...,2017-01-12 00:00:00,176035.0,NLST,NaN,1282631.0
42176,2017,2,14145,./code/data/newly_add_transcript/Transcript_Di...,2017-06-06 00:00:00,18564.0,AAOI,NaN,1158114.0
42177,2014,3,90885,./code/data/newly_add_transcript/Transcript_Di...,2014-09-08 00:00:00,163678.0,EVRI,NaN,1318568.0
42178,2015,3,14159,./code/data/newly_add_transcript/Transcript_Di...,2015-09-09 00:00:00,18487.0,FEYE,NaN,1370880.0
42179,2014,4,10107,./code/data/newly_add_transcript/Transcript_Di...,2014-05-29 00:00:00,12141.0,MSFT,NaN,789019.0


In [17]:
df_2.shape

(2237, 9)

In [18]:
#df_3 = df_2[df_2.path.str.contains('Earnings Call')]

In [19]:
#就算没有earnings call在title 但其实还是earnings call
#df_3_f = df_2[~df_2.path.str.contains('Earnings Call')]

# process raw data

In [ ]:
# test for debug
# remove footnote & header & copyright

In [201]:
#p = df_2.path.sample().values[0]
p = './code/data/newly_add_transcript/Transcript_Digest/Global Cash Access Holdings, Inc., Movie Merger Sub, Inc., Multimedia Games Holding Company, Inc. - MA Call.pdf'

In [202]:
#p = df_2.path.sample().values[0]
print(p)
parsed = parser.from_file(p)

./code/data/newly_add_transcript/Transcript_Digest/Global Cash Access Holdings, Inc., Movie Merger Sub, Inc., Multimedia Games Holding Company, Inc. - MA Call.pdf


In [ ]:
parsed

In [204]:
c = parsed["content"]
c_list = c.split('\n')

In [205]:
c_list = [c for c in c_list if len(c.strip())>0]

In [206]:
#after read couple pdf file, I find the notes should be removed:
n1 = parsed["metadata"]['resourceName']
n2 = 'Copyright © 2019 S&P Global Market Intelligence, a division of S&P Global Inc. All Rights reserved.'
n3 = 'https://marketintelligence.spglobal.com/'
n4 = 'spglobal.com/marketintelligence'
n5 = '© 2019 S&P Global Market Intelligence.'

In [207]:
n1 = n1.lower()
n2 = n2.lower()
n3 = n3.lower()
n4 = n4.lower()
n5 = n5.lower()

In [208]:
res = []
for i in c_list:
    i = i.lower()
    if ((fuzz.token_set_ratio(n1,i) > 75) | (fuzz.ratio(n2,i) > 75) | (fuzz.ratio(n3,i) > 75)|
        (fuzz.ratio(n4,i) > 75) | (fuzz.ratio(n5,i) > 75)):
        pass
    else:
        res.append(i)

In [214]:
s = 'everi holdings inc. m&a call |  sep 08, 2014'
s = s.lower()
fuzz.token_set_ratio(n1,s)

61

In [215]:
print(n1)

b'global cash access holdings, inc., movie merger sub, inc., multimedia games holding company, inc. - ma call.pdf'


In [216]:
from collections import defaultdict, Counter

In [212]:
#Compare original text and processed text
d = dict(Counter(c_list))
r = dict(Counter(res))

In [213]:
sorted(r.items(), key=lambda item: item[1])

[('1', 1),
 ('everi holdings inc. nyse:evri', 1),
 ('m&a call', 1),
 ('monday, september 08, 2014 10:00 pm gmt', 1),
 ('contents', 1),
 ('2', 1),
 ('table of contents', 1),
 ('call participants .................................................................................. 3',
  1),
 ('presentation .................................................................................. 4',
  1),
 ('question and answer .................................................................................. 6',
  1),
 ('call participants', 1),
 ('executives', 1),
 ('former chief executive officer,', 1),
 ('president and director', 1),
 ('analysts', 1),
 ('sterne agee & leach inc.,', 1),
 ('craig-hallum capital group llc,', 1),
 ('stifel, nicolaus & company,', 1),
 ('incorporated, research division', 1),
 ('presentation', 1),
 ('hello, everyone. thank you for standing by, and welcome to the global cash access holdings incorporated',
  1),
 ('corporate conference call. [operator instructions] this 

In [112]:
sorted(d.items(), key=lambda item: item[1])

[('1', 1),
 ('Netlist, Inc. OTCPK:NLST', 1),
 ('Company Conference ', 1),
 ('Thursday, January 12, 2017 1:00 PM GMT', 1),
 ('Contents', 1),
 ('2', 1),
 ('Table of Contents', 1),
 ('Call Participants .................................................................................. 3',
  1),
 ('Presentation .................................................................................. 4',
  1),
 ('Question and Answer .................................................................................. 6',
  1),
 ('spglobal.com/marketintelligence 3', 1),
 ('Call Participants', 1),
 ('EXECUTIVES', 1),
 ('Co-Founder, President, CEO &', 1),
 ('Executive Chairman', 1),
 ('ANALYSTS', 1),
 ('Needham & Company, LLC,', 1),
 ('Roth Capital Partners, LLC,', 1),
 ('spglobal.com/marketintelligence 4', 1),
 ("Let's get started. Good morning. And welcome again to the 19th Annual Needham Growth Conference. My",
  1),
 ("name is Rich Kugele, I'm the Enterprise Data Center Infrastructure Hardware and So

In [217]:
res

['1',
 'everi holdings inc. nyse:evri',
 'm&a call',
 'monday, september 08, 2014 10:00 pm gmt',
 'contents',
 '2',
 'table of contents',
 'call participants .................................................................................. 3',
 'presentation .................................................................................. 4',
 'question and answer .................................................................................. 6',
 'everi holdings inc. m&a call |  sep 08, 2014',
 'call participants',
 'executives',
 'ram v. chary',
 'former chief executive officer,',
 'president and director',
 'randy l. taylor',
 'executive vp & cfo',
 'todd valli',
 'analysts',
 'david brian bain',
 'sterne agee & leach inc.,',
 'research division',
 'george frederick sutton',
 'craig-hallum capital group llc,',
 'research division',
 'jeffrey bronchick',
 'cove street capital, llc',
 'john kimbrough davis',
 'stifel, nicolaus & company,',
 'incorporated, research division',
 'la

In [114]:
# test for debug
# organize presetnation part and Q&A part
# In preserntation, remove operater speech
# In Q&A, organize analyst & exicutive speech

In [218]:
def match_check(x, checklist, fuzzy_thresh=70):
    res = any(map(lambda c: fuzz.ratio(x,c)>fuzzy_thresh, checklist))
    return res

In [219]:
# input text should be the a list after remvoed the notes
a = [aa.lower() for aa in res]
# if 'no q&a session for this event' in ''.join(a): 
#     return None

init = a.index('table of contents')
a = a[init:]

x=a.index('executives') # 找到人名清单
y=a.index('analysts') # 有没有可能存在没有记录的清单?
z=a.index('presentation')
time=''.join(a[:x])
anss=a[x+1:y] + ['unidentified company representative']
# anss_short = [re.sub(' - .+', '', anss_1) for anss_1 in anss]
askk=a[y+1:z]+['unidentified analyst - unidentified company'] + ['unidentified analyst'] + ['unidentified company'] + ['unknown analyst']
# askk_short =  [re.sub(' - .+', '', ask) for ask in askk]
opt = ['operator']
#     print('ans', anss)
#     print('ask', askk)

In [226]:
anss

['ram v. chary',
 'former chief executive officer,',
 'president and director',
 'randy l. taylor',
 'executive vp & cfo',
 'todd valli',
 'unidentified company representative']

In [227]:
askk

['david brian bain',
 'sterne agee & leach inc.,',
 'research division',
 'george frederick sutton',
 'craig-hallum capital group llc,',
 'research division',
 'jeffrey bronchick',
 'cove street capital, llc',
 'john kimbrough davis',
 'stifel, nicolaus & company,',
 'incorporated, research division',
 'larry haverty',
 'matthew j. kempler',
 'sidoti & company, llc',
 'todd joseph eilers',
 'eilers & krejcik gaming, llc',
 'everi holdings inc. m&a call |  sep 08, 2014',
 'unidentified analyst - unidentified company',
 'unidentified analyst',
 'unidentified company',
 'unknown analyst']

In [228]:

# 找到presentation部分
pre = 0
p = 0
while (fuzz.ratio(a[p],'Presentation'.lower()) < 80):
    p+=1 
pre = p

# 找到 QA 开始的部分
qa = 0
q = pre
while ((fuzz.ratio(a[q],'Question and Answer'.lower()) < 80) and (fuzz.ratio(a[q],'Question-and-Answer-Session'.lower()) < 80) and (fuzz.ratio(a[q],'Question-&-nswer-Session'.lower()) < 80)):
    q+=1 
qa = q

#对全文进行标注speaker
b=[x for x in a if x != ''] 
b_orig = [x for x in res[init:] if x != '']

statu_list = []
idx_list = []
ques_count = 0
for b_idx, bb in enumerate(b):

    ask_name_check = match_check(bb, askk)
    ans_name_check = match_check(bb, anss)
    opt_name_check = match_check(bb, opt)
    if ask_name_check == True:
        statu_list.append('a') #analyst
        idx_list.append(b_idx)
    elif ans_name_check == True:
        statu_list.append('e') #exicutive
        idx_list.append(b_idx)
    elif opt_name_check == True:
        statu_list.append('o') #operator
        idx_list.append(b_idx)

#分割为presentation part 和 QA part
pre_idx =  [i for i in idx_list if ((i < qa) & (i > pre))] #_idx标注了第几句话是人名
pre_statu = statu_list[idx_list.index(pre_idx[0]):idx_list.index(pre_idx[-1])+1] #_status标注了这个人名的属性：operater、 anlayst、exicutes

qa_idx = [i for i in idx_list if (i > qa)]
qa_statu = statu_list[idx_list.index(qa_idx[0]):]

#分割presentation part
#operater的speech需要被剔除
pre_res = []
end = -1
for i, idx in enumerate(pre_idx):
    if idx > end:
        start_speaker = pre_statu[i]
        start = idx
        length = len(pre_statu)-1
        while (i<length) and (pre_statu[i]==pre_statu[i+1]):
            i += 1
        if i == length:
            end = qa #如果在最后一个对话block pre的结束应该在QAsesstion 前
        else:
            end = pre_idx[i+1] #如果不在最后一个对话block，这个block的结束应该在下一个block开始前
            
        if start_speaker == 'e': # if the speaker is exicute
            pre_res = a[start:end] # only keep the exicutive's speech
pre_res = '\n'.join(pre_res)

In [229]:
pre_res

"randy l. taylor\nexecutive vp & cfo\nthank you, ram, and hello, everyone. as ram mentioned, we are very excited about this gaming-relevant\ntransformational opportunity. the transaction is expected to be immediately accretive to both earnings\nand free cash flow as of the closing date of this acquisition. the combination is expected to achieve\napproximately $30 million of run rate synergies within 12 months. on a pro forma basis, we will generate\nabout $800 million in revenues and approximately $217 million adjusted ebitda, including the synergies.\nboth companies have predictable and relatively stable cash flows. the significant cash flow generation\nincludes the utilization of gca's net operating loss of over $100 million over the next 3 years. the\ncombined entity will also benefit from gca's tax-deductible goodwill through 2019. the cash flow should\nenable the company to quickly reduce the debt incurred as part of this transaction.\nnow the terms of the transaction. gca will ac

In [242]:
statu_list

['a',
 'e',
 'e',
 'e',
 'e',
 'e',
 'e',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'o',
 'e',
 'e',
 'e',
 'a',
 'e',
 'e',
 'e',
 'e',
 'a',
 'o',
 'a',
 'a',
 'e',
 'e',
 'a',
 'a',
 'e',
 'e',
 'a',
 'a',
 'e',
 'e',
 'a',
 'a',
 'e',
 'e',
 'a',
 'o',
 'a',
 'a',
 'e',
 'e',
 'a',
 'a',
 'e',
 'e',
 'a',
 'a',
 'e',
 'e',
 'a',
 'a',
 'a',
 'e',
 'e',
 'o',
 'a',
 'a',
 'e',
 'e',
 'a',
 'a',
 'e',
 'e',
 'a',
 'o',
 'a',
 'a',
 'e',
 'e',
 'a',
 'a',
 'e',
 'e',
 'a',
 'a',
 'e',
 'e',
 'a',
 'a',
 'e',
 'e',
 'a',
 'a',
 'a',
 'e',
 'e',
 'a',
 'a',
 'e',
 'e',
 'a',
 'a',
 'e',
 'e',
 'o',
 'a',
 'a',
 'a',
 'e',
 'e',
 'a',
 'a',
 'e',
 'e',
 'o',
 'a',
 'a',
 'e',
 'e',
 'a',
 'a',
 'a',
 'e',
 'e',
 'a',
 'a',
 'e',
 'e',
 'a',
 'a',
 'e',
 'e',
 'o',
 'a',
 'e',
 'e',
 'a',
 'a',
 'e',
 'e',
 'a',
 'e',
 'e',
 'o',
 'a',
 'e',
 'e',
 'o',
 'a',
 'e',
 'o',
 'a']

In [244]:
#分割QA part
#构建成 Q_A结构
#QA比较复杂在于 需要确定 有一个Q再有一个A
res_list = []
for idx in np.arange(len(qa_statu)):
#         print(idx)
    if qa_statu[idx] == 'a': #once get a question 

        if (idx-1 >=0) and qa_statu[idx-1] == 'a': #如果下一个说话人还是analyst
            pass
        else: #如果不是 说明一个问题表达结束
            ques_count += 1
            res_dict = dict()
            res_dict['question_'+str(ques_count)] = dict()
            res_dict['question_'+str(ques_count)] ['ask_name'] = b[qa_idx[idx]]

            #q_asker = b[idx_list[idx]]

            que_start_index = qa_idx[idx] + 1
            #que_end_index = idx_list[idx+1]
            idx_next_que = idx+1
            #print('q_idxnext', idx_next_que)
            while ((idx_next_que < len(qa_idx)-1) and (qa_statu[idx_next_que] == 'a')): #如果连续两个analyst发言
                res_dict['question_'+str(ques_count)] ['ask_name'] += ', '+b[qa_idx[idx_next_que]]
                idx_next_que += 1
            if idx_next_que > len(qa_idx)-1:
                que_end_index = len(b)-1
            else:
                que_end_index = qa_idx[idx_next_que]
            res_dict['question_'+str(ques_count)] ['question'] = b_orig[que_start_index:que_end_index]
            #q_text = b[que_start_index:que_end_index]
        #res_dict['question_'+str(ques_count)] = {“”}
            
            
        
        
    if qa_statu[idx] == 'e': #如果是exicutive发言
        #assert ques_count >=1 #必须有analyst发言之后
        if (idx-1 >=0) and qa_statu[idx-1] == 'e': #对于同一问题 如果已有exicutive回答过
            #res_dict['question_'+str(ques_count)] ['ans_name'] += b[idx_list[idx]]
            pass
        else:
            res_dict['question_'+str(ques_count)] ['ans_name'] = b[qa_idx[idx]]
            ans_start_index = qa_idx[idx] + 1
            #ans_end_index = idx_list[idx+1]
            idx_next_ans = idx+1
            while ((idx_next_ans < len(qa_idx)-1) and (qa_statu[idx_next_ans] == 'e')):
                res_dict['question_'+str(ques_count)] ['ans_name'] += ', '+ b[qa_idx[idx_next_ans]]
                idx_next_ans += 1

            if idx_next_ans >  len(qa_idx)-1: #如果对话在最后一个block
                ans_end_index = len(b)-1
            else: ans_end_index = qa_idx[idx_next_ans]
            res_dict['question_'+str(ques_count)] ['answer'] = b_orig[ans_start_index:ans_end_index]
            res_list.append(res_dict)
# qa_res = dict()
# end = -1
# que_count = 0
# for i, idx in enumerate(qa_idx):
#     que_res = []
#     ans_res = []
#     if idx > end:
#         start_speaker = qa_statu[i]
#         start = idx
#         length = len(qa_statu)-1
#         while (i<length) & (qa_statu[i]==qa_statu[i+1]):
#             i += 1
#         if i == length:
#             end = len(a) #如果在最后一个对话block block的结束应该在文章最后
#         else:
#             end = qa_idx[i+1] #如果不在最后一个对话block，这个block的结束应该在下一个block开始前
#         temp_idx = qa_idx[qa_idx.index(start):qa_idx.index(end)+1]
            
#         if start_speaker == 'e': # if the speaker is exicute
#             ans_res = b_orig[start:end] # keep the exicutive's speech to answer
#             qa_res['question_'+str(que_count)]['answer_name'] = [a[i] for i in temp_idx]    #exicutive's names are the qa index 
#             qa_res['question_'+str(que_count)]['answer'] = '\n'.join(ans_res)
#         if start_speaker == 'a':# if the speaker is analyst
#             que_count += 1 #question comes firstly
#             que_res = b_orig[start:end]
#             qa_res['question_'+str(que_count)] = dict()
#             qa_res['question_'+str(que_count)]['question_name'] = [a[i] for i in temp_idx]    #question's names are the qa index 
#             qa_res['question_'+str(que_count)]['question'] = '\n'.join(que_res)


In [245]:
res[124]

'operator'

In [246]:
qa_statu[-1]

'a'

In [247]:
len(qa_statu)

124

In [248]:
idx_next_que

124

In [251]:
b_orig[pre]

'presentation'

In [252]:
b_orig[qa]

'question and answer'

In [253]:
res[qa_idx[1]]

'forward.'

In [254]:
res[qa_idx[0]]

'are fully committed to help our customers during the transition period. together, the combined entity will'

In [255]:
res_list

[{'question_125': {'ans_name': 'ram v. chary, former chief executive officer, president and director',
   'answer': ['former chief executive officer, president and director',
    "it's all cost."],
   'ask_name': 'david brian bain, sterne agee & leach inc., research division',
   'question': ['sterne agee & leach inc., research division',
    'guys, on the cost synergy front, you noted the $30 million, actually, overall synergies. is that all cost',
    'synergies or is there some top line there?']}},
 {'question_126': {'ans_name': 'ram v. chary, former chief executive officer, president and director',
   'answer': ['former chief executive officer, president and director',
    "well, as recently years have gone on within the casino operators organizations and they've tried to adapt",
    "to changing market dynamics, david, you'd be surprised at how close those decision points have come",
    'within their organizations. and so we have a nice, very small but complementary overlap in te

In [ ]:
askk

['brian william ruttenbur',
 'crt capital group llc, research',
 'division',
 'charles edward cerankosky',
 'northcoast research partners, llc',
 'gautam j. khanna',
 'cowen and company, llc,',
 'research division',
 'gregory arnold konrad',
 'jefferies llc, research division',
 'james andrew chartier',
 'monness, crespi, hardt & co.,',
 'inc., research division',
 "john o'neil",
 'imperial capital, llc, research',
 'division',
 'scott w. hamann',
 'keybanc capital markets inc.,',
 'research division',
 'presentation',
 'unidentified analyst - unidentified company',
 'unidentified analyst',
 'unidentified company']

In [ ]:
statu_list

['e',
 'e',
 'e',
 'e',
 'e',
 'e',
 'e',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'o',
 'e',
 'e',
 'e',
 'e',
 'e',
 'e',
 'o',
 'a',
 'a',
 'e',
 'e',
 'a',
 'a',
 'e',
 'e',
 'o',
 'a',
 'a',
 'e',
 'e',
 'a',
 'a',
 'e',
 'e',
 'o',
 'a',
 'a',
 'e',
 'e',
 'a',
 'a',
 'e',
 'e',
 'e',
 'e',
 'e',
 'e',
 'o',
 'a',
 'a',
 'e',
 'e',
 'a',
 'a',
 'e',
 'e',
 'o',
 'a',
 'a',
 'e',
 'e',
 'a',
 'a',
 'e',
 'e',
 'o',
 'a',
 'a',
 'e',
 'e',
 'a',
 'a',
 'e',
 'e',
 'e',
 'e',
 'o',
 'a',
 'a',
 'e',
 'e',
 'a',
 'a',
 'e',
 'e',
 'o',
 'e',
 'e',
 'o']

In [ ]:
pre_statu

['o']

In [ ]:
qa_statu

['o',
 'a',
 'a',
 'e',
 'e',
 'a',
 'a',
 'e',
 'e',
 'o',
 'a',
 'a',
 'e',
 'e',
 'a',
 'a',
 'e',
 'e',
 'o',
 'a',
 'a',
 'e',
 'e',
 'a',
 'a',
 'e',
 'e',
 'e',
 'e',
 'e',
 'e',
 'o',
 'a',
 'a',
 'e',
 'e',
 'a',
 'a',
 'e',
 'e',
 'o',
 'a',
 'a',
 'e',
 'e',
 'a',
 'a',
 'e',
 'e',
 'o',
 'a',
 'a',
 'e',
 'e',
 'a',
 'a',
 'e',
 'e',
 'e',
 'e',
 'o',
 'a',
 'a',
 'e',
 'e',
 'a',
 'a',
 'e',
 'e',
 'o',
 'e',
 'e',
 'o']

In [ ]:
qa_idx

[298,
 300,
 301,
 305,
 306,
 322,
 323,
 328,
 329,
 347,
 349,
 350,
 355,
 356,
 376,
 377,
 382,
 383,
 405,
 407,
 408,
 412,
 413,
 416,
 417,
 419,
 420,
 428,
 429,
 443,
 444,
 450,
 452,
 453,
 457,
 458,
 470,
 471,
 474,
 475,
 477,
 479,
 480,
 486,
 487,
 510,
 511,
 516,
 517,
 536,
 538,
 539,
 542,
 543,
 570,
 571,
 575,
 576,
 594,
 595,
 599,
 601,
 602,
 607,
 608,
 612,
 613,
 617,
 618,
 640,
 642,
 643,
 647]

## wrap up previous code to function

In [131]:
def process_pdf(p):
    parsed = parser.from_file(p)

    c = parsed["content"]
    c_list = c.split('\n')

    c_list = [c for c in c_list if len(c.strip())>0]

    n1 = parsed["metadata"]['resourceName']
    n2 = 'Copyright © 2019 S&P Global Market Intelligence, a division of S&P Global Inc. All Rights reserved.'
    n3 = 'https://marketintelligence.spglobal.com/'
    n4 = 'spglobal.com/marketintelligence'
    n5 = '© 2019 S&P Global Market Intelligence.'

    n1 = n1.lower()
    n2 = n2.lower()
    n3 = n3.lower()
    n4 = n4.lower()
    n5 = n5.lower()

    res = []
    for i in c_list:
        i = i.lower()
        if ((fuzz.token_set_ratio(n1,i) > 70) | (fuzz.token_set_ratio(n2,i) > 70) | (fuzz.token_set_ratio(n3,i) > 70)|
            (fuzz.token_set_ratio(n4,i) > 70) | (fuzz.token_set_ratio(n5,i) > 70)):
            pass
        else:
            res.append(i)

    return res


In [169]:
def match_check(x, checklist, fuzzy_thresh=70):
    res = any(map(lambda c: fuzz.ratio(x,c)>fuzzy_thresh, checklist))
    return res

In [256]:
def organize_trascript(res):
    a = [aa.lower() for aa in res]

    init = a.index('table of contents')
    # print('init',init)
    a = a[init:]
    # print(len(a))

    x=a.index('executives') # 找到人名清单
    y=a.index('analysts') # 有没有可能存在没有记录的清单?
    z = a.index('presentation')
    #time=''.join(a[:x])
    anss=a[x+1:y] + ['unidentified company representative'] + ['unknown representative']
    # anss_short = [re.sub(' - .+', '', anss_1) for anss_1 in anss]
    askk=a[y+1:z]+['unidentified analyst - unidentified company'] + ['unidentified analyst'] + ['unidentified company'] + ['unknown analyst']
    # askk_short =  [re.sub(' - .+', '', ask) for ask in askk]
    opt = ['operator']    

    # 找到presentation部分
    pre = 0
    p = 0
    while (fuzz.ratio(a[p],'Presentation'.lower()) < 80):
        p+=1 
    pre = p

    # 找到 QA 开始的部分
    qa = 0
    q = pre
    while ((fuzz.ratio(a[q],'Question and Answer'.lower()) < 80) and (fuzz.ratio(a[q],'Question-and-Answer-Session'.lower()) < 80) and (fuzz.ratio(a[q],'Question-&-nswer-Session'.lower()) < 80)):
        q+=1 
    qa = q

    #对全文进行标注speaker
    b=[x for x in a if x != ''] 
    b_orig = [x for x in res[init:] if x != '']

    statu_list = []
    idx_list = []
    ques_count = 0
    for b_idx, bb in enumerate(b):
        # print(bb)
        # global C
        # global D
        C = askk
        D = anss

        ask_name_check = match_check(bb, askk)
        ans_name_check = match_check(bb, anss)
        opt_name_check = match_check(bb, opt)
        if ask_name_check == True:
            statu_list.append('a') #analyst
            idx_list.append(b_idx)
        elif ans_name_check == True:
            statu_list.append('e') #exicutive
            idx_list.append(b_idx)
        elif opt_name_check == True:
            statu_list.append('o') #operator
            idx_list.append(b_idx)

    #分割为presentation part 和 QA part
    # global A
    # global B
    # print(idx_list)
    # print(statu_list)
    pre_idx =  [i for i in idx_list if ((i < qa) & (i > pre))] #_idx标注了第几句话是人名
    A = pre_idx
    B = idx_list
    pre_statu = statu_list[idx_list.index(pre_idx[0]):idx_list.index(pre_idx[-1])+1] #_status标注了这个人名的属性：operater、 anlayst、exicutes

    qa_idx = [i for i in idx_list if (i > qa)]
    qa_statu = statu_list[idx_list.index(qa_idx[0]):]

    #分割presentation part
    #operater的speech需要被剔除
    pre_res = []
    end = -1
    for i, idx in enumerate(pre_idx):
        if idx > end:
            start_speaker = pre_statu[i]
            start = idx
            length = len(pre_statu)-1
            while (i<length) and (pre_statu[i]==pre_statu[i+1]):
                i += 1
            if i == length:
                end = qa #如果在最后一个对话block pre的结束应该在QAsesstion 前
            else:
                end = pre_idx[i+1] #如果不在最后一个对话block，这个block的结束应该在下一个block开始前
                
            if start_speaker == 'e': # if the speaker is exicute
                pre_res = a[start:end] # only keep the exicutive's speech
    pre_res = '\n'.join(pre_res)


    res_list = []
    for idx in np.arange(len(qa_statu)):
    #         print(idx)
        if qa_statu[idx] == 'a': #once get a question 

            if (idx-1 >=0) and qa_statu[idx-1] == 'a': #如果下一个说话人还是analyst
                pass
            else: #如果不是 说明一个问题表达结束
                ques_count += 1
                res_dict = dict()
                res_dict['question_'+str(ques_count)] = dict()
                res_dict['question_'+str(ques_count)] ['ask_name'] = b[qa_idx[idx]]

                #q_asker = b[idx_list[idx]]

                que_start_index = qa_idx[idx] + 1
                #que_end_index = idx_list[idx+1]
                idx_next_que = idx+1
                #print('q_idxnext', idx_next_que)
                while ((idx_next_que < len(qa_idx)-1) and (qa_statu[idx_next_que] == 'a')): #如果连续两个analyst发言
                    res_dict['question_'+str(ques_count)] ['ask_name'] += ', '+b[qa_idx[idx_next_que]]
                    idx_next_que += 1
                if idx_next_que > len(qa_idx)-1:
                    que_end_index = len(b)-1
                else:
                    que_end_index = qa_idx[idx_next_que]
                res_dict['question_'+str(ques_count)] ['question'] = b_orig[que_start_index:que_end_index]
                #q_text = b[que_start_index:que_end_index]
            #res_dict['question_'+str(ques_count)] = {“”}
                
                
            
            
        if qa_statu[idx] == 'e': #如果是exicutive发言
            assert ques_count >=1 #必须有analyst发言之后
            if (idx-1 >=0) and qa_statu[idx-1] == 'e': #对于同一问题 如果已有exicutive回答过
                #res_dict['question_'+str(ques_count)] ['ans_name'] += b[idx_list[idx]]
                pass
            else:
                res_dict['question_'+str(ques_count)] ['ans_name'] = b[qa_idx[idx]]
                ans_start_index = qa_idx[idx] + 1
                #ans_end_index = idx_list[idx+1]
                idx_next_ans = idx+1
                while ((idx_next_ans <= len(qa_idx)-1) and (qa_statu[idx_next_ans] == 'e')):
                    res_dict['question_'+str(ques_count)] ['ans_name'] += ', '+ b[qa_idx[idx_next_ans]]
                    idx_next_ans += 1

                if idx_next_ans >  len(qa_idx)-1: #如果对话在最后一个block
                    ans_end_index = len(b)-1
                else: ans_end_index = qa_idx[idx_next_ans]
                res_dict['question_'+str(ques_count)] ['answer'] = b_orig[ans_start_index:ans_end_index]
                res_list.append(res_dict)


    return pre_res, res_list


In [257]:
def convert_tb_qa(res_list, path):
    r_1_tb = pd.DataFrame(columns = ['ask_name','question','ans_name','answer'])
    for r in res_list:
        r_1_temp = pd.DataFrame(r).T
        #print(r_1_temp)
        r_1_tb = r_1_tb.append( r_1_temp)
    r_1_tb['path'] = path
    return r_1_tb

In [258]:
def convert_tb_pre(pre_res, path):
    text = pre_res
    tb = pd.DataFrame({'path':[path], 'text': [text]})
    return tb


In [259]:
#test
A = None
B = None
C = None
D = None
qa_tb_fl = pd.DataFrame(columns = ['ask_name','question','ans_name','answer'])
pre_tb_fl = pd.DataFrame(columns = ['path','text'])
c = 0
fail = []
for p in df_2.iloc[0:10,:].path:
    c += 1
    if c%100 ==0:
        print(c)
    # try:
    print(p)
    res = process_pdf(p)
    pre_res, res_list = organize_trascript(res)
    pre_tb = convert_tb_pre(pre_res, p)

    qa_tb = convert_tb_qa(res_list, p)
    qa_tb_fl = qa_tb_fl.append(qa_tb)
    pre_tb_fl = pre_tb_fl.append(qa_tb)
    # except:
    #     fail.append(p)

./code/data/newly_add_transcript/Transcript_Digest/Netlist, Inc. Presents at 19th Annual Needham Growth Conference, Jan-12-2017 0800 AM.pdf
./code/data/newly_add_transcript/Transcript_Digest/Applied Optoelectronics, Inc. Presents at Stifel 2017 Technology, Internet  Media Conference, Jun-06-2017 0835 AM.pdf
./code/data/newly_add_transcript/Transcript_Digest/Global Cash Access Holdings, Inc., Movie Merger Sub, Inc., Multimedia Games Holding Company, Inc. - MA Call.pdf
./code/data/newly_add_transcript/Transcript_Digest/FireEye, Inc. Presents at Citis 2015 Global Technology Conference, Sep-09-2015 0930 AM.pdf
./code/data/newly_add_transcript/Transcript_Digest/Microsoft Corporation, Salesforce.com, Inc - Special Call.pdf
./code/data/newly_add_transcript/Transcript_Digest/The Gap, Inc., Period Ending Jan 03, 2015 Pre Recorded Sales Trading Statement Call, Jan 08, 2015.pdf


ValueError: ignored

In [197]:
p

'./code/data/newly_add_transcript/Transcript_Digest/Netlist, Inc. Presents at 19th Annual Needham Growth Conference, Jan-12-2017 0800 AM.pdf'

In [198]:
pd.DataFrame({'path':[p], 'text':[pre_res]})


,path,text
0,./code/data/newly_add_transcript/Transcript_Di...,"co-founder, president, ceo & executive chairma..."


In [167]:
C

['richard john kugele',
 'needham & company, llc,',
 'research division',
 'sujeeva desilva',
 'roth capital partners, llc,',
 'research division',
 'unknown analyst',
 'unidentified analyst - unidentified company',
 'unidentified analyst',
 'unidentified company',
 'unknown analyst']

In [168]:
D

['chun k. hong',
 'co-founder, president, ceo &',
 'executive chairman',
 'unidentified company representative',
 'unknown representative']

In [172]:
A

[]

In [173]:
B

[]

In [ ]:
qa_tb

In [32]:
qa_tb_fl

,ask_name,question,ans_name,answer


In [22]:
fail

['./code/data/newly_add_transcript/Transcript_Digest/Netlist, Inc. Presents at 19th Annual Needham Growth Conference, Jan-12-2017 0800 AM.pdf',
 './code/data/newly_add_transcript/Transcript_Digest/Applied Optoelectronics, Inc. Presents at Stifel 2017 Technology, Internet  Media Conference, Jun-06-2017 0835 AM.pdf',
 './code/data/newly_add_transcript/Transcript_Digest/Global Cash Access Holdings, Inc., Movie Merger Sub, Inc., Multimedia Games Holding Company, Inc. - MA Call.pdf',
 './code/data/newly_add_transcript/Transcript_Digest/FireEye, Inc. Presents at Citis 2015 Global Technology Conference, Sep-09-2015 0930 AM.pdf',
 './code/data/newly_add_transcript/Transcript_Digest/Microsoft Corporation, Salesforce.com, Inc - Special Call.pdf',
 './code/data/newly_add_transcript/Transcript_Digest/The Gap, Inc., Period Ending Jan 03, 2015 Pre Recorded Sales Trading Statement Call, Jan 08, 2015.pdf',
 './code/data/newly_add_transcript/Transcript_Digest/The Mosaic Company Presents at Credit Suiss

In [ ]:
qa_tb_fl = pd.DataFrame(columns = ['ask_name','question','ans_name','answer'])
pre_tb_fl = pd.DataFrame(columns = ['path','text'])
c = 0
fail = []
for p in df_2.path:
    c += 1
    if c%100 ==0:
        print(c)
    try:
        res = process_pdf(p)
        pre_res, res_list = organize_trascript(res)
        pre_tb = convert_tb_pre(pre_res, p)
        qa_tb = convert_tb_qa(res_list, p)
        qa_tb_fl = qa_tb_fl.append(qa_tb)
        pre_tb_fl = pre_tb_fl.append(qa_tb)
    except:
        fail.append(p)

In [ ]:
raw_qa = pd.read_csv('./code/data/Conf_features/Conf_transcripts_new_parsed.csv')  

In [ ]:
raw.head()

,Unnamed: 0,question_num,ans_name,answer,ask_name,path,question,time_text,time_text_len,ticker,rid,encoded_unique_ticker_ts,report_day
0,0,question_1,"george paleologou, will kalutycz","['Yes, generally speaking, a lot of the rebala...",sabahat khan,/backup_disk/conference_calls/PRBZF/4073438-pr...,['Thanks and good afternoon. Can you maybe tal...,Premium Brands Holding Corp. (OTC:PRBZF) Q1 20...,97,PRBZF,0.0,63130.0,2017-05-15
1,1,question_2,will kalutycz,"['Yes, it’s kind of a mixture, the Sandwich pl...",sabahat khan,/backup_disk/conference_calls/PRBZF/4073438-pr...,"['Alright, thanks. And then you mentioned that...",Premium Brands Holding Corp. (OTC:PRBZF) Q1 20...,97,PRBZF,1.0,63130.0,2017-05-15
2,2,question_3,will kalutycz,"['Yes, again, I mean we deal in all commoditie...",sabahat khan,/backup_disk/conference_calls/PRBZF/4073438-pr...,['Great. And then just if you could give us up...,Premium Brands Holding Corp. (OTC:PRBZF) Q1 20...,97,PRBZF,2.0,63130.0,2017-05-15
3,3,question_5,george paleologou,['Thanks George.'],george doumet,/backup_disk/conference_calls/PRBZF/4073438-pr...,"['Hey, good afternoon guys and congrats on the...",Premium Brands Holding Corp. (OTC:PRBZF) Q1 20...,97,PRBZF,3.0,63130.0,2017-05-15
4,4,question_6,george paleologou,"['Yes, actually, it was more -- it was not too...",george doumet,/backup_disk/conference_calls/PRBZF/4073438-pr...,['Just looking at Alberta first year-over-year...,Premium Brands Holding Corp. (OTC:PRBZF) Q1 20...,97,PRBZF,4.0,63130.0,2017-05-15


In [ ]:
raw.shape

(2369700, 13)